### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from urllib.parse import quote_plus
import re

In [5]:
# Set environment variables from the .env in the local environment    

def key_check(key_path=None):
    try:
        reply=load_dotenv(key_path,verbose=True,override=True)   
        assert reply , 'Dotenv is not found'
        nyt_api_key = os.getenv("NYT_API_KEY")
        tmdb_api_key = os.getenv("TMDB_API_KEY")
        assert nyt_api_key is not None, 'NYT_API_KEY not found in .env file'
        assert tmdb_api_key is not None, 'TMDB_API_KEY not found in .env file'
        responce=requests.get(f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={nyt_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate nyt_api_key {nyt_api_key} code {responce.status_code}'
        responce=requests.get(f'https://api.themoviedb.org/3/movie/11?api_key={tmdb_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {tmdb_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
        return(False)
    else:
        print('All keys loaded correctly')
        return (True)
my_env_path= 'C:\src\\ai\data-sourcing-challenge\.data-sourcing_challenge.env'
if key_check(my_env_path):
    nyt_api_key = os.getenv("NYT_API_KEY")
    tmdb_api_key = os.getenv("TMDB_API_KEY")
else:
    print ('fix Keys and rerun')

All keys loaded correctly


### Access the New York Times API

In [6]:
# Sample from web page
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=new+york+times&page=2&sort=oldest&api-key=your-api-key 

#  Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = quote_plus('section_name:"Movies" AND type_of_material:"Review" AND headline:"love"')

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = quote_plus("headline,web_url,snippet,source,keywords,pub_date,byline,word_count")

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=(f'{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}&api-key={nyt_api_key}&page=')
display (query_url)

'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name%3A%22Movies%22+AND+type_of_material%3A%22Review%22+AND+headline%3A%22love%22&begin_date=20130101&end_date=20230531&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&sort=newest&api-key=eFnt18GxLtCn0CZ0E23Hqrb5BIJ9e5ce&page='

In [7]:
# Create an empty list to store the reviews
reviews_list =[]
review_page=0
# loop through pages 0-19
start_page = 0
end_page = 20

for page in range(start_page,end_page):
    # Construct the query URL for the current page
    page_url = f'{query_url}{page}'
    # Attempt to make a "GET" request and parse the JSON response
    reviews = requests.get(page_url).json()
    try:
        # Check if the "docs" list is empty; if so, print a message and exit the loop
        if (reviews["response"]["docs"]):
            print(f'Checked page  {page}')
            # print ('sleep 12 seconds')
            time.sleep(12)                 
            # Add a twelve second pause between requests to adhere to API query limits
        else: 
            raise ValueError (f'No results on page {page} {review_page + 1} reviews found, stopping.')
    except ValueError as e:
        # Handle the case where no documents are found
        print(e)
        break
    except Exception as e:
        # Handle potential errors in the request or data processing
        print(f'An error occurred: {e}')
        break        
        # Otherwise, process each article in "docs"
    for review in reviews["response"]["docs"]:
        reviews_list.append(review)
        review_page += 1
        # print (f'review {review_page}' )
        
    

Checked page  0
Checked page  1
Checked page  2
Checked page  3
Checked page  4
Checked page  5
Checked page  6
Checked page  7
Checked page  8
Checked page  9
Checked page  10
Checked page  11
Checked page  12
Checked page  13
Checked page  14
Checked page  15
Checked page  16
Checked page  17
Checked page  18
Checked page  19


In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df=pd.json_normalize(reviews_list)
review_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


The original lambda function finds some titles but with a few changes it finds all but one. I used my version. the regular expression misses many cases. this is important later when the title is used to look up movie id in the tmdb. 


```python
lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
```
I rewrote this using re.search and the following regular expression

```python
    lambda st: re.search(r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)", st).group(1) /
            if re.search(r"\u2018(.+?)\u2019", st) else None)
```

In [10]:
# Extract the title from the "headline.main" column and
# 
# Regular expression to match text enclosed by ‘ and ’
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
 
review_df['title'] = review_df['headline.main'].apply(
    lambda st: re.search(r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)", st).group(1)
            if re.search(r"\u2018(.+?)\u2019", st) else None)

#playing with column display width
pd.set_option('display.max_colwidth', None)
display (review_df[['title','headline.main']])
pd.set_option('display.max_colwidth', 50)
display (review_df[['title','headline.main']])

,title,headline.main
0,The Attachment Diaries,"‘The Attachment Diaries’ Review: Love, Sick"
1,What’s Love Got to Do With It?,Review: ‘What’s Love Got to Do With It?’ Probably a Lot
2,You Can Live Forever,‘You Can Live Forever’ Review: Do You Love Me Now?
3,A Tourist’s Guide to Love,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip
4,Other People’s Children,‘Other People’s Children’ Review: True Romance
...,...,...
195,The Other Half,Review: A Combustible Pair Find Love in ‘The Other Half’
196,The Ottoman Lieutenant,"Review: Love as the World Wars, in ‘The Ottoman Lieutenant’"
197,Love & Taxes,Review: It’s All Mirth and Taxes in ‘Love & Taxes’
198,"Everybody Loves Somebody,","Review: ‘Everybody Loves Somebody,’ a Rom-Com With Bite"


,title,headline.main
0,The Attachment Diaries,"‘The Attachment Diaries’ Review: Love, Sick"
1,What’s Love Got to Do With It?,Review: ‘What’s Love Got to Do With It?’ Proba...
2,You Can Live Forever,‘You Can Live Forever’ Review: Do You Love Me ...
3,A Tourist’s Guide to Love,‘A Tourist’s Guide to Love’ Review: A Wearying...
4,Other People’s Children,‘Other People’s Children’ Review: True Romance
...,...,...
195,The Other Half,Review: A Combustible Pair Find Love in ‘The O...
196,The Ottoman Lieutenant,"Review: Love as the World Wars, in ‘The Ottoma..."
197,Love & Taxes,Review: It’s All Mirth and Taxes in ‘Love & Ta...
198,"Everybody Loves Somebody,","Review: ‘Everybody Loves Somebody,’ a Rom-Com ..."


In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# this copy make it possible for me to rerun this without having to start from scratch
review_mod_df = review_df.copy(deep=True)

# Fix the "keywords" column by converting cells from a list to a string
review_mod_df['keywords'] = review_mod_df['keywords'].apply(extract_keywords)
# display(lmr_keyword_df[['title','keywords']].head(3))
#
#play time with style
styled_subset_df=review_mod_df.loc[:4,['title','keywords']]
styled_df = styled_subset_df.style.set_table_styles({
    'title': [{'selector': '',
                'props': [('width', '200px'), ('text-align', 'right')]}],
    'keywords': [{'selector': '',
                'props': [('width', '700px'), ('text-align', 'left')]}]
}, overwrite=False).hide(axis=0)
# display(styled_df)
display (review_mod_df)
display (styled_df)
del styled_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"

title,keywords
The Attachment Diaries,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;"
What’s Love Got to Do With It?,"subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;"
You Can Live Forever,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);"
A Tourist’s Guide to Love,"subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;"
Other People’s Children,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);"


In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = review_mod_df['title'].to_list()
print("Top 5 Titles:\n" + json.dumps(titles_list[:5], indent= 4, ensure_ascii=False))
display (titles_list)

Top 5 Titles:
[
    "The Attachment Diaries",
    "What’s Love Got to Do With It?",
    "You Can Live Forever",
    "A Tourist’s Guide to Love",
    "Other People’s Children"
]


['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journa

### Access The Movie Database API

In [13]:
# Prepare The Movie Database query
url_query   = "https://api.themoviedb.org/3/search/movie?query="
url_detail  = "https://api.themoviedb.org/3/movie/"

While researching the tmdb api doc I found that the rate limiting isnt as strict.

https://developer.themoviedb.org/docs/rate-limiting
While our legacy rate limits have been disabled for some time, 
we do still have some upper limits to help mitigate needlessly high bulk scraping. 
They sit somewhere in the 50 requests per second range. 
This limit could change at any time so be respectful of the service we have built and respect the 429 if you receive one.

I modified the code to look for status code of 429 and pause if they get one. 

original code block for assignemnt is

```python
if (request_counter % 50) == 0:
    print (f'Sleeping at {request_counter} requests')
    time.sleep(1)
    request_counter += 1
else:
    # Add 1 to the request counter
    request_counter += 1
```

In [14]:
# Create an empty list to store the results
tmbd_movies_list=[]

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
 
    # Perform a "GET" request for The Movie Database
    response=requests.get(f'{url_query}{title}"&api_key={tmdb_api_key}')
    if response.status_code == 429:
        print (f'Recieved respnse code 429. Sleeping 1 second')
        time.sleep(1)
        
    data = response.json()   
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found. 
    
    try:            
        if data['results'] != []:
            movie_id = data['results'][0]['id']
            # Make a request for the full movie details
            # Execute "GET" request with url
            detail_response = requests.get(f"{url_detail}{movie_id}?api_key={tmdb_api_key}")
            if response.status_code == 429:
                print (f'Recieved respnse code 429. Sleeping 1 second')
                time.sleep(1)              
            movie_detail = detail_response.json()
            # Extract the genre names into a list
            genres_list = [genre['name'] for genre in movie_detail['genres']]
            # Extract the spoken_languages' English name into a list
            spoken_languages = [spoken_language['english_name'] for spoken_language in movie_detail['spoken_languages']]
            # Extract the production_countries' name into a list
            production_countries = [production_countries['name'] for production_countries in movie_detail['production_countries']]        
            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            tmbd_movies_list.append({
                # 'movie_id'        : movie_id,
                'title'           : title,
                'original_title'  : (movie_detail['original_title']),
                'budget'          : (movie_detail['budget']),
                'original_language'  : (movie_detail['original_language']),
                'homepage'        : (movie_detail['homepage']),
                'overview'        : (movie_detail['overview']),
                'popularity'      : (movie_detail)['popularity'],
                'runtime'         : (movie_detail['runtime']),
                'revenue'         : (movie_detail['revenue']),
                'release_date'    : (movie_detail['release_date']),
                'vote_average'    : (movie_detail['vote_average']),
                'vote_count'      : (movie_detail['vote_count']),
                'gendre_list'     : genres_list,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries})
                    
            # Print out the title that was found
            print (f'Found {title}')
        else: 
            raise ValueError (f"{title} not found.")   
    except ValueError as e:
        # Handle the case where no documents are found
        print(e)
    except requests.exceptions.RequestException as e:
    # This catches all exceptions that are requests-related
    # Including connection errors, timeouts, etc.
        print("A network error occurred. Please try again later.")
        print(e)



Found The Attachment Diaries
Found What’s Love Got to Do With It?
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found Th

In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(tmbd_movies_list[:5],indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.418,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "gendre_list": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
      

In [16]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmbd_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,gendre_list,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,21.338,109,10898395,2023-01-26,6.062,162,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,6.641,32,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",22.981,96,0,2023-04-21,6.329,155,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.963,104,84178,2022-09-21,6.900,178,"[Drama, Comedy]","[English, French]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.356,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
195,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",22.372,111,413844,2017-03-10,6.173,245,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"
196,Love & Taxes,Romantik Hırsız,0,tr,https://www.netflix.com/title/81689799,After learning that the art thief she has been...,180.545,99,0,2024-03-14,6.735,85,"[Romance, Action, Drama]",[Turkish],[Turkey]
197,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,en,,"On the surface, Clara Barron seems to have it ...",10.107,102,0,2017-02-10,6.600,158,"[Romance, Comedy]","[English, Spanish]",[Mexico]


### Merge and Clean the Data for Export

In [25]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,review_mod_df, on= 'title', how='inner')
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,21.338,109,10898395,2023-01-26,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",22.981,96,0,2023-04-21,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.963,104,84178,2022-09-21,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.356,103,0,2016-12-02,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
197,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",22.372,111,413844,2017-03-10,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
198,Love & Taxes,Romantik Hırsız,0,tr,https://www.netflix.com/title/81689799,After learning that the art thief she has been...,180.545,99,0,2024-03-14,...,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
199,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,en,,"On the surface, Clara Barron seems to have it ...",10.107,102,0,2017-02-10,...,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


 wanted to be able to get all columns that were lists and use that in a loop. 
 I could have created a list f columns that contained lists. 
 This seemed cleaner. The extra column byline person is deleted in the next step.
 This might not work everwhere but seems to work here and was a nice exersize. 

In [18]:
# Remove list brackets and quotation marks on the columns containing lists
# added remove () for byline.person. just for practice cleaning data
# Create a list of the columns that need fixing
clean_merged_df=merged_df.copy(deep=True)
columns_to_fix = [col for col in clean_merged_df.columns if clean_merged_df[col].map(lambda x: isinstance(x, list)).any()]
print (columns_to_fix)
display (clean_merged_df[columns_to_fix].head())
# Create a list of characters to remove
def clean_list_string(s):
    if isinstance(s, list):
        # Convert list to string and remove unwanted characters
        return str(s).strip('[]').strip('{}').replace('"', '').replace("'", "")
    return s

# Apply the cleaning directly, no need for preliminary list check
for col in columns_to_fix:
    clean_merged_df[col] = clean_merged_df[col].apply(clean_list_string)

# Display the fixed DataFrame
display (clean_merged_df[columns_to_fix].head())

['gendre_list', 'spoken_languages', 'production_countries', 'byline.person']


,gendre_list,spoken_languages,production_countries,byline.person
0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],"[{'firstname': 'Jeannette', 'middlename': None..."
1,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]","[{'firstname': 'Jeannette', 'middlename': None..."
2,"[Drama, Romance]","[English, French]","[Canada, United States of America]","[{'firstname': 'Elisabeth', 'middlename': None..."
3,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],"[{'firstname': 'Elisabeth', 'middlename': None..."
4,"[Drama, Comedy]","[English, French]",[France],"[{'firstname': 'Manohla', 'middlename': None, ..."


,gendre_list,spoken_languages,production_countries,byline.person
0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,"firstname: Jeannette, middlename: None, lastna..."
1,"Romance, Comedy","English, Portuguese, Urdu","France, United Kingdom","firstname: Jeannette, middlename: None, lastna..."
2,"Drama, Romance","English, French","Canada, United States of America","firstname: Elisabeth, middlename: None, lastna..."
3,"Romance, Comedy","English, Vietnamese",United States of America,"firstname: Elisabeth, middlename: None, lastna..."
4,"Drama, Comedy","English, French",France,"firstname: Manohla, middlename: None, lastname..."


In [19]:
# Drop "byline.person" column

merged_df.drop(columns=['byline.person'], inplace=True)

In [22]:
# Delete duplicate rows and reset index
duplicates = clean_merged_df[clean_merged_df.duplicated('title')]
display (duplicates)
clean_merged_df = clean_merged_df.drop_duplicates('title',keep='first').reset_index(drop=True)
clean_merged_df.head(20)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.418,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"firstname: Jeannette, middlename: None, lastna...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,21.338,109,10898395,2023-01-26,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"firstname: Jeannette, middlename: None, lastna...",None
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",50.557,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"firstname: Elisabeth, middlename: None, lastna...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",22.981,96,0,2023-04-21,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"firstname: Elisabeth, middlename: None, lastna...",None
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.963,104,84178,2022-09-21,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"firstname: Manohla, middlename: None, lastname...",None
5,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,15.317,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"firstname: Brandon, middlename: None, lastname...",None
6,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.355,95,77145,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"firstname: Glenn, middlename: None, lastname: ...",None
7,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,13.264,116,3395595,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"firstname: Manohla, middlename: None, lastname...",None
8,Your Place or Mine,Your Place or Mine,0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,24.250,109,0,2023-02-10,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"firstname: Amy, middlename: None, lastname: Ni...",None
9,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,,An intimate portrait of a community fighting t...,1.203,85,0,2023-02-03,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"firstname: Concepción, middlename: None, lastn...",None


In [21]:
# Export data to CSV without the index
clean_merged_df.to_csv('tmdb_reviews.csv',index=False, encoding='utf-8')